In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

C:\Users\mook\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
xy = np.loadtxt('./data/stock_daily.csv',dtype=np.float32,delimiter=',')

In [3]:
"""
매일 최신순으로 정렬된 주식 데이터를 가지고 7일치씩 끊어서 train시키고 
8일차의 close값을 예측하여라
""" 

'\n매일 최신순으로 정렬된 주식 데이터를 가지고 7일치씩 끊어서 train시키고 \n8일차의 close값을 예측하여라\n'

## 뒤집기 -> 시간순으로 배열

In [4]:
xy=xy[::-1] #사람은 배워야해

In [5]:
# dc = data[:,:] 
# for i in range(int(len(data)/2)):
#     dc[i] = data[-i]
#     dc[-i] = data[i]

## 스케일링

In [6]:
def MinMaxScalar(data):
    bunja = data-np.min(data,0)
    bunmo = np.max(data,0) - np.min(data,0)
    return bunja/(bunmo+1e-10)

In [7]:
a =np.array([[10,20,30,40,50], 
             [0,2,3,4,9],
             [4,10,28,0,39],
             [6,15,5,39,29]])
a =MinMaxScalar(a)
a

array([[1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.        , 0.        , 0.        , 0.1       , 0.        ],
       [0.4       , 0.44444444, 0.92592593, 0.        , 0.73170732],
       [0.6       , 0.72222222, 0.07407407, 0.975     , 0.48780488]])

In [8]:
xy= MinMaxScalar(xy)

In [9]:
data_dim = 5
hidden_dim = 16
output_dim = 1 #결과값은 다음날 close값 하나임
seq_length = 7
learning_rate = 0.1

# X,Y구분

In [10]:
x = xy
y = xy[:,[-1]]

In [18]:
np.array(x).shape

(732, 5)

In [11]:
np.array(y).shape

(732, 1)

In [17]:
dataX = []
dataY = []
for i in range(0, len(y)-seq_length):
    _x = x[i:i+seq_length]
    _y = y[i+seq_length]
    dataX.append(_x)
    dataY.append(_y)

In [21]:
np.array(dataX).shape

(725, 7, 5)

In [13]:
batch_size = len(dataX)
batch_size

725

In [14]:
batch_size = len(xy)-seq_length
batch_size

725

In [16]:
train_size = int(len(dataY)*0.7)
test_size = len(dataY)- train_size

In [22]:
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

In [24]:
trainX.shape

(507, 7, 5)

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, shape=[None,1])
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True,
                                  activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell,X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim,
                                         activation_fn=None)
loss = tf.reduce_mean(tf.reduce_sum(tf.square(Y_pred-Y)))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)

target = tf.placeholder(tf.float32, [None,1])
prediction = tf.placeholder(tf.float32, [None,1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(target-prediction)))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
   l,_ = sess.run([loss, train], feed_dict={X:dataX, Y:dataY})
   if step % 100 == 0:
       print ("step:",step,"loss:",l)

In [ ]:
test_prediction = sess.run(Y_pred, feed_dict={X:testX})
print ("RMSE:",sess.run(rmse, feed_dict={target:testY, prediction:test_prediction}))

In [ ]:
plt.plot(testY)
plt.plot(test_prediction)